In [ ]:
#Se deben instalar los paquetes que se utilizan a continuación y descargar un controlador de Chrome para visualizar mejor 
#los resultados obtenidos de acuerdo con las estructuras de la página.

In [8]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from requests import session
import time
import pandas as pd

#Se utiliza Selenium debido a que incorpora herramientas para automatizar la espera en la carga de la página, evitando los TimeSleep

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [10]:
#SCOPUS

#Se abre la página de SINAB, se hace visible a través de ChromeDriver y se espera hasta que se carguen los elementos por completo.
pagina = webdriver.Chrome(r"/Applications/chromedriver")
pagina.get("https://login.ezproxy.unal.edu.co/login?url=https://www.scopus.com/results/results.uri?sort=plf-f&src=s&sid=02019aebd600d4583c56339c9a68afaa&sot=a&sdt=a&sl=23&s=SOURCE-ID+%2812600154771%29&origin=sourceinfo&zone=CSCYpreview&txGid=e19dfeb01248ac0a4f988644499ba4cd")
espera = WebDriverWait(pagina,20)
success = False

#Se ingresan los datos del usuario para acceder a la página principal de Scopus.
while not success:
    try:
        #Se encuentran los elementos de la página web en donde se ponen el usuario y la contraseña.
        usuario = pagina.find_element_by_name("user");contraseña = pagina.find_element_by_name("pass")
        
        #Se ponen los datos del usuario en los campos encontrados y se da click en el botón de 'Login'
        usuario.send_keys(username);contraseña.send_keys(password)
        login_attempt = espera.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/form/input[2]")))
        login_attempt.submit() 

        success = True
        
    #A continuación se redirecciona en caso de que el tiempo de espera por la página se exceda.
    except TimeoutException:
        pagina.get("https://login.ezproxy.unal.edu.co/login?url=https://www.scopus.com/results/results.uri?sort=plf-f&src=s&sid=02019aebd600d4583c56339c9a68afaa&sot=a&sdt=a&sl=23&s=SOURCE-ID+%2812600154771%29&origin=sourceinfo&zone=CSCYpreview&txGid=e19dfeb01248ac0a4f988644499ba4cd")
    
    #Se lee archivo CSV como una tabla de Pandas que contiene el AuthorID de los profesores.
    file_scopus = pd.read_csv('Scopus.txt',  sep = ',', header=0)
    
    
    general_scopus = []
    for i in range(0,len(file_scopus)):
        #Se encuentra el AuthorID del profesor en la posición i.
        word = file_scopus.iloc[i,0]  
        pagina.get('https://www-scopus-com.ezproxy.unal.edu.co/authid/detail.uri?authorId='+str(word)+'')
        
        #Se para el código por 3 segundos mientras carga el contenido de la página.
        time.sleep(3)
        
        #Se pasa de 20 publicaciones a 200 por página para leer solo el contenido de una URL y no tener que iterar sobre
        #las páginas
        objeto = espera.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="resultsPerPage-button"]/span[1]')))
        objeto.click()
        objeto = espera.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ui-id-12"]')))
        time.sleep(10)
        objeto.click()
    
        #Se extrae el número de publicaciones que tiene el profesor en total.
        numero = espera.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="authorDetailsDocumentsByAuthor"]/div/div[2]/span'))).text

        profe_scopus= []
        for i in range(int(numero)):
            temp = []
            #De acuerdo con la estructura del XPath que contiene el nombre del artículo y el año de publicación, se itera
            #sobre él para obtener todos los posibles de acuerdo con el número total de publicaciones.
            articulo_scopus = espera.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="resultDataRow'+str(i)+'"]/td[1]/a'))).text
            articulo = articulo_scopus.lower()
            time.sleep(3)
            año_scopus = espera.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="resultDataRow'+str(i)+'"]/td[3]'))).text
            temp.append(articulo);temp.append(año_scopus);profe_scopus.append(temp)
        general_scopus.append(profe_scopus)

#Se almacena la información del profesor Juan David Velasquez Henao como una tabla de pandas.        
JDV_Scopus = pd.DataFrame(general_scopus[0],columns=["Artículo","Año"])

WebDriverException: Message: unknown error: Element <span class="ui-selectmenu-icon ui-icon btn-primary btn-icon ico-navigate-down flexDisplay flexAlignCenter flexJustifyCenter"></span> is not clickable at point (181, 407). Other element would receive the click: <html lang="en_US">...</html>
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.4 x86_64)


In [ ]:
JDV_Scopus

---

In [ ]:
#RESEARCH_GATE
#Es necesario obtener usuario y contraseña para hacer busquedas y/o modificar las URL y obtener información de valor.
username = input('Ingrese su correo: ')
password = input('Ingrese su contraseña: ')

#Se ingresa a la página principal 
pagina = webdriver.Chrome(r"C:\Users\Area\Desktop\M4\chromedriver.exe")
pagina.get("https://www.researchgate.net/login")
espera = WebDriverWait(pagina, 20)

#Se ingresan los datos de usuario y contraseña
success = False
while not success:
    try:
        usuario = pagina.find_element_by_xpath('//*[@id="input-login"]');contraseña = pagina.find_element_by_xpath('//*[@id="input-password"]')
        usuario.send_keys(username);contraseña.send_keys(password)

        obj_clickable = espera.until(EC.element_to_be_clickable((By.CLASS_NAME,'nova-c-button__label')))
        obj_clickable.click()
        success = True

    except TimeoutException:
        pagina.get("https://www.researchgate.net/login")
        
    
    #Se lee archivo TXT como una tabla de Pandas que contiene el usuario asociado a cada perfil(URL)
    file_RGate = pd.read_csv('ResearchGate.txt',  sep = ',', header=0);general_RGate = []
    
    #Se itera sobre los perfiles 
    for i in range(0,len(file_RGate)):
        profe_RGate = [];word = file_RGate.iloc[i,0] 
        #Se itera sobre las páginas del perfil correspondiente.
        for k in range(1,8):
            pagina.get('https://www.researchgate.net/profile/'+str(word)+'/publications?sorting=newest&page='+str(k)+'')
            time.sleep(2)
            y = pagina.find_elements_by_class_name('nova-v-publication-item__stack-item')
            for j in range(0,len(y),3):
                temp = [];articulo = y[j].text;articulo_RGate= articulo.lower();año_RGate = y[j+1].text
                tipo = año_RGate.split("\n")[0]
                temp.append(articulo_RGate);temp.append(tipo);profe_RGate.append(temp)
        general_RGate.append(profe_RGate)
        
#Se almacena la información del profesor Juan David Velasquez Henao como una tabla de pandas. 
JDV_RGate = pd.DataFrame(general_RGate[0],columns=["Artículo","Tipo"])

In [ ]:
#CvLAC
#Para acceder al perfil de los profesores no es necesario acceder a una cuenta.
pagina = webdriver.Chrome(r"C:\Users\Area\Desktop\M4\chromedriver.exe")
espera = WebDriverWait(pagina,20)

#Se lee archivo TXT como una tabla de Pandas que contiene el códigoID asociado a cada perfil(URL), es decir, este identificador
#es único para cada perfil y se encuentra al final de la URL del perfil.
file = open('CvLAC.txt', 'r').readlines()
x = [z.replace('\n', '') for z in file]

general=[]
#Se itera sobre cada perfil.
for j in range(len(file)):
    profe= []
    gg = 'http://scienti.colciencias.gov.co:8081/cvlac/visualizador/generarCurriculoCv.do?cod_rh='
    pagina.get(gg+x[j])
    o = pagina.find_element_by_xpath('/html/body/div/div[3]/table/tbody/tr[30]/td/table').text
    #La información de las publicaciones se encuentran en un bloque de texto, por lo tanto es necesario modificarlo para obtener
    #los datos que se necesitan
    xx = o.split('Artículo');y = [i.split('"') for i in xx]
    
    for f in range(2,len(y)):
        uno = []
        articulo1 = y[f][1]; articulo= articulo1.lower()
        año = y[f][2].split(",")[1]
        uno.append(articulo); uno.append(año);profe.append(uno)
    general.append(profe)
    
#Se almacena la información del profesor Juan David Velasquez Henao como una tabla de pandas.    
JDV_CV = pd.DataFrame(general[0],columns=["Artículo","Año"])

In [ ]:
#Teniendo en cuenta la información del profesor Juan David Velasquez Henao, se busca crear una lista de publicaciones única 
#que contenga los artículo y el año de publicación.

#Se filtran SOLO los artículos de la lista en ResearchGate
y = JDV_RGate.groupby('Tipo')
y.groups.keys()

#Se comparan las tres tablas para encontrar los registros únicos.
g = pd.merge(JDV_Scopus,JDV_RGate.loc[y.groups['Article']], how='right', on=['Artículo'])
Table = pd.merge(g,JDV_CV, how='right', on=['Artículo'])

#Se eliminan registros duplicados.
Table_2= Table.drop_duplicates()

Table_2

In [5]:
#Datos para logearse a través de la plataforma SINAB hacia Scopus.
username = 'vavasquezhe'
password = '2ooRIQRBfdvv'